In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv('sample_text.csv')

In [4]:
df.shape

(8, 2)

In [6]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


###  Step 1 : Create source embeddings for the text column

In [8]:
from sentence_transformers import SentenceTransformer

In [11]:
encoder = SentenceTransformer("all-mpnet-base-v2")


In [12]:
vectors = encoder.encode(df.text)

In [13]:
vectors.shape

(8, 768)

In [15]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [17]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [18]:
index.add(vectors)

In [19]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000013FEBCDBE70> >

### Step 4 : Encode search text using same encoder and normalize the output vector

In [20]:
search_query = "I want to buy a polo t-shirt"

In [21]:
vec = encoder.encode(search_query)

In [23]:
vec.shape

(768,)

In [24]:
svec = np.array(vec).reshape(1, -1)
svec.shape

(1, 768)

### Step 5: Search for similar vector in the FAISS index created

In [27]:
distances, I = index.search(svec, k = 2)

In [28]:
distances

array([[1.384484 , 1.4039096]], dtype=float32)

In [29]:
I

array([[3, 2]])

In [30]:
I.tolist()

[[3, 2]]

In [31]:
row_indices = I.tolist()[0]

In [32]:
row_indices

[3, 2]

In [33]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [34]:
search_query

'I want to buy a polo t-shirt'